In [1]:
from Tools import *
# import all tool function from tool set

In [2]:
from PIL import Image, ImageFont, ImageDraw
import numpy as np

from random import randint, choice
from math import sin, cos, radians, fabs

import os,sys
dir_path = os.path.abspath('.')
print(dir_path)
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

G:\Python_Lab\Proj


In [3]:
def GenerateOneChar(status=None, character=None, corp_r=20, ran_bias=5):
    """
    status == 0 front stand
    status == 1 head stand
    radius generally < 50
    ran_bias: random bias used in generating a single character
    character: Chinese character to produce. is string-type
    """
    bwh = 160 # width and height of background
    iwh = (81, 90) # width and height of image
    fontsize = 72
    
    if status == None:
        status = 0
    if status == 0:
        angle = random.uniform(-30, 30) 
    else:
        angle = random.uniform(150, 210) 
    
    if character == None:
        character = GetA_GB2312_Char()
    
    im = Image.new("RGBA", iwh, (0,0,0,0)) # black font
    
    global dir_path 
    font = ImageFont.truetype(dir_path + "\\Kai-Simplified-Bold.ttf", fontsize)
    
    ch = ImageDraw.Draw(im)
    ch.fontmode = "1"
    ch.text((0, 0), character, font=font, fill="#000000") # fill with black
    #plt.imshow(im)
    #plt.show()
    im = CutOffRegion(im) # reserve only character region
    #plt.imshow(im)
    #plt.show()
    
    # roration
    fore = im.rotate(angle, resample=Image.BICUBIC,expand=1)
    width, height = fore.size
    # zooming
    scale = np.random.uniform(0.95, 1.05)
    fore = fore.resize((int(width * scale), int(height * scale)), Image.ANTIALIAS)
    width, height = fore.size
    
    #plt.imshow(fore)
    #plt.show()
    ##
    background = Image.new("RGB", (bwh, bwh), (255,255,255,255)) #white background
    background.paste(fore, (bwh//2 - width // 2 + randint(-ran_bias, ran_bias),\
                            bwh//2 - height // 2 + randint(-ran_bias, ran_bias)), fore)
    
    return character,background.crop((bwh//2 - corp_r, bwh//2 - corp_r,\
                                      bwh//2 + corp_r, bwh//2 + corp_r))

In [18]:
import sys,time
def Generate(nums=10,corp_r=20,ran_bias=5,status=-1,path=None,random_c=False,outC=False):
    """
    nums: the number of characters to produce
    corp_r: radius of corping
    ran_bias: random bias used in generating a single character
    status: headstand or not
    random_c: true if use randomly corping in generating a single character
    outC: true if want to write characters into files. At this time, path should not be None
    """
    
    X = np.zeros((nums,40,40,1))
    if status == 0:
        y = np.zeros(nums)
    elif status == 1:
        y = np.ones(nums)
    else:
        y = np.zeros(nums)
        for i in range(nums):
            #randomly generate statuses
            y[i] = np.random.binomial(1, 0.5)
    if path == None:
        path = dir_path
        
    for i in range(nums):
        #cited and modified from http://hovertree.com/h/bjaf/opmktjog.htm
        #print the progress
        p = int(((i+1)/nums)*100)
        String = '>'*(p//2)+' '*((100-p-1)//2)
        sys.stdout.write('\rPocessing: '+String+'[%s%%]'%(p))
        sys.stdout.flush()
        #############
        if random_c == False:
            Char,CharImg = GenerateOneChar(status=y[i],corp_r=corp_r)
        else:
            Char,CharImg = GenerateOneChar(status=y[i],corp_r=randint(20, 50))
        
        CharImg = CharImg.resize((40,40))
        #CharImg2 = np.asarray(CharImg.convert('RGB').convert('L'),dtype='float64')
        #CharImg2[CharImg2 <= 180] = 0
        #CharImg2[CharImg2 > 180] = 255
        #plt.imshow(Image.fromarray(CharImg2))
        #plt.show()
        CharImg = np.asarray(CharImg.convert('RGB').convert('L'),dtype='float64')
        CharImg[CharImg <= 180] = 0
        CharImg[CharImg > 180] = 255
        
        if outC == True:
            filename = path+'\\'+"%06d"%(i)+Char+str(int(y[i]))+'.bmp'
            #print(filename)
            Image.fromarray(CharImg).convert('RGB').save(filename)
        
        X[i,:,:,:] = CharImg[:,:,None]
        
    return X, y

In [19]:
Xr,yr=Generate(nums=131072,corp_r=35,ran_bias=10,status=-1,path='./tr',random_c=False,outC=True)

Pocessing: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>[100%]

In [22]:
np.save("Xr.npy", Xr)
np.save("yr.npy", yr)

In [23]:
Xs,ys=Generate(nums=2048,corp_r=35,ran_bias=10,status=-1,path='./ts',random_c=False,outC=True)

Pocessing: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>[100%]

In [24]:
np.save("Xs.npy", Xs)
np.save("ys.npy", ys)